In [1]:
chinese_dataset = 'article_9k.txt'

In [2]:
with open(chinese_dataset,'r') as f:
    CHINESE_CHARATES = f.read()

In [3]:
CHINESE_CHARATES[:40]

'此外自本周6月12日起除小米手机6等15款机型外其余机型已暂停更新发布含开发版体'

In [4]:
import pinyin

In [5]:
def chinese_to_pinyin(character):
    return pinyin.get(character,format='strip',delimiter=' ')

In [6]:
CHINESE_CHARATES_COPYS = chinese_to_pinyin(CHINESE_CHARATES)

In [7]:
len(CHINESE_CHARATES_COPYS)

129433034

In [8]:
del(CHINESE_CHARATES)

In [9]:
import re

In [10]:
def tokens(text):
    return re.findall('[a-z]+',text.lower())

In [11]:
from collections import Counter,defaultdict

In [12]:
PINYIN_COUNT = Counter(tokens(CHINESE_CHARATES_COPYS))

In [13]:
PINYIN_ALL = tokens(CHINESE_CHARATES_COPYS)

In [14]:
def correct(word):
    candidates = (known(edits0(word)) or
                  known(edits1(word)) or
                  known(edits2(word)) or
                  [word])
    return max(candidates,key=PINYIN_COUNT.get)

In [15]:
def known(words):
    return {w for w in words if w in PINYIN_COUNT}

def edits0(word):
    return {word}

def edits2(word):
    return { e2 for e1 in edits1(word) for e2 in edits1(e1)}

In [16]:
def splits(word):
    return [(word[:i],word[i:])
            for i in range(len(word)+1)]
def edits1(word):
    pairs = splits(word)
    deletes = [a+b[1:] for (a,b) in pairs if b]
    transposes = [a+b[1]+b[0]+b[2:] for (a,b) in pairs if len(b) > 1]
    replaces = [a+c+b[1:] for (a,b) in pairs for c in alphabet if b]
    inserts = [a+c+b for (a,b) in pairs for c in alphabet]
    return set(deletes + transposes + replaces + inserts)

In [17]:
alphabet = 'abcdefghijklmnopqrstuvwxyz'

In [18]:
def correct_sequence_pinyin(text_pinyin):
    return ' '.join(map(correct, text_pinyin.split()))

In [19]:
correct_sequence_pinyin('wo xiang shagn qinng hua da xue')

'wo xiang shang qing hua da xue'

In [20]:
PINYIN_ALL[:10]

['ci', 'wai', 'zi', 'ben', 'zhou', 'yue', 'ri', 'qi', 'chu', 'xiao']

In [21]:
PINYIN_COUNT.most_common(10)

[('shi', 860634),
 ('de', 809887),
 ('n', 691114),
 ('yi', 682478),
 ('ji', 645276),
 ('guo', 430042),
 ('zhong', 409418),
 ('zhi', 398612),
 ('xin', 359619),
 ('li', 355441)]

In [22]:
def prob_1(word):
    return PINYIN_COUNT[word] / len_article

In [23]:
len_PINYIN_ALL = len(PINYIN_ALL)

In [24]:
PINYIN_ALL_2 = [''.join(PINYIN_ALL[i:i+2]) for i in range(len_PINYIN_ALL-2)]

In [25]:
PINYIN_ALL_3 = [''.join(PINYIN_ALL[i:i+3]) for i in range(len_PINYIN_ALL-3)]

In [26]:
PINYIN_COUNT_2 = Counter(PINYIN_ALL_2)

In [27]:
PINYIN_COUNT_3 = Counter(PINYIN_ALL_3)

In [28]:
len_PINYIN_ALL_2 = len(PINYIN_ALL_2)

In [29]:
len_PINYIN_ALL_3 = len(PINYIN_ALL_3)

In [30]:
def prob_2(word1,word2):
    if word1 + word2 in PINYIN_COUNT_2: return PINYIN_COUNT_2[word1+word2] / PINYIN_COUNT[word2]
    else: return 1/len_PINYIN_ALL_2

In [31]:
def prob_3(word1,word2,word3):
    if word1 + word2 + word3 in PINYIN_COUNT_3: return PINYIN_COUNT_3[word1+word2+word3] / PINYIN_COUNT_2[word2+word3]
    else: return 1/len_PINYIN_ALL_3

In [33]:
def get_probablity_2_gram(words):
    
    sentence_pro = 1
    
    for i, word in enumerate(words[:-1]):
        next_1 = words[i+1]
        
        probability = prob_2(word, next_1)
        
        sentence_pro *= probability
    sentence_pro = probability * (PINYIN_COUNT[next_1] / len_PINYIN_ALL_2)
    return sentence_pro

In [36]:
def get_probablity_3_gram(words):
    
    sentence_pro = 1
    
    for i, word in enumerate(words[:-2]):
        next_1 = words[i+1]
        next_2 = words[i+2]
        
        probability = prob_3(word, next_1, next_2)
        
        sentence_pro *= probability
    sentence_pro = probability * (PINYIN_COUNT_2[next_1+next_2] / len_PINYIN_ALL_3)
    return sentence_pro

In [37]:
yuanyin = ['a','i','e','o','u','v']

In [38]:
#元音位置的list
def find_yuanyin_position(string):
    yuanyin_position = []
    for i in yuanyin:
        i_list = [j.start() for j in re.finditer(str(i), string)]
        yuanyin_position = yuanyin_position + i_list
        yuanyin_position = sorted(yuanyin_position)
    return yuanyin_position

In [39]:
#把一个完整的拼音切出来
def cut_pinyin(string):
    #所有元音的位置
    yuanyin_position = find_yuanyin_position(string)
    #找到第一个元音的位置
    min_position = yuanyin_position[0]
    #如果只有一个拼音且只有一个元音
    if len(yuanyin_position) == 1:
        return string
    #如果只有一个拼音且有两个元音
    elif len(yuanyin_position) == 2 and (yuanyin_position[0] + 1 == yuanyin_position[1]):
        return string
    elif len(yuanyin_position) == 3 and (yuanyin_position[0] + 1 == yuanyin_position[1]) and (yuanyin_position[0] + 2 == yuanyin_position[2]):
        return string
    else:
        #如果第一个拼音有一个元音
        if (min_position + 1) not in yuanyin_position:
            #找到第三个元音也是第二个拼音的元音
            second_pinyin_yuanyin = yuanyin_position[1]
            if string[second_pinyin_yuanyin-2] in ['z','c','s']:
                return string[:second_pinyin_yuanyin-2]
            else:return string[:second_pinyin_yuanyin-1]
        #如果第一个拼音有两个元音且第二个拼音不仅仅是一个字母
        elif (min_position + 1) in yuanyin_position and (min_position + 2) not in yuanyin_position:
            #找到第三个元音也是第二个拼音的元音
            second_pinyin_yuanyin = yuanyin_position[2]
            #判断第二个拼音前面有几个辅音属于第二个拼音
            if string[second_pinyin_yuanyin-2] in ['z','c','s']:
                return string[:second_pinyin_yuanyin-2]
            else:return string[:second_pinyin_yuanyin-1]
        #如果第一个拼音有两个元音且第二个拼音仅仅是一个字母
#         elif (min_position + 1) in yuanyin_position and (min_position + 2) in yuanyin_position:
#             second_pinyin_yuanyin = yuanyin_position[2]
#             return string[:second_pinyin_yuanyin]
        #如果第一个拼音有三个元音
        elif (min_position + 1) in yuanyin_position and (min_position + 2) in yuanyin_position:
            second_pinyin_yuanyin = yuanyin_position[2]
            return string[:second_pinyin_yuanyin+1]

In [95]:
#把这个完整的拼音切分成多种可能性 如：zhuang: zhu zhuang
def cut_one_pinyin(one_pinyin):
    #每个拼音的可能性
    all_pinyin = []
    if (one_pinyin[-1] in ['i','u','v']) or (len(find_yuanyin_position(one_pinyin)) == 1):
        list1 = []
        list1.append(one_pinyin)
        all_pinyin.append(list1)
        return all_pinyin

    elif len(find_yuanyin_position(one_pinyin)) == 2:
        #这里是zhuang的第一个元音位置
        first_yuanyin_position = find_yuanyin_position(one_pinyin)[0]
        #找出两个可能
        list1 = []
        list2 = []
        list1.append(one_pinyin[:first_yuanyin_position+1])
        list1.append(one_pinyin[first_yuanyin_position+1:])
        list2.append(one_pinyin)
        all_pinyin.append(list1)
        all_pinyin.append(list2)
        return all_pinyin
    elif len(find_yuanyin_position(one_pinyin)) == 3:
        #这里是shuai的第一个元音位置
        first_yuanyin_position = find_yuanyin_position(one_pinyin)[0]
        #找出四个可能
        list1 = []
        list2 = []
        list3 = []
        list4 = []
        list1.append(one_pinyin[:first_yuanyin_position+1])
        list1.append(one_pinyin[first_yuanyin_position+1:])
        list2.append(one_pinyin[:first_yuanyin_position+2])
        list2.append(one_pinyin[first_yuanyin_position+2:])
        list3.append(one_pinyin[:first_yuanyin_position+1])
        list3.append(one_pinyin[first_yuanyin_position+1])
        list3.append(one_pinyin[first_yuanyin_position+2])
        list4.append(one_pinyin)
        all_pinyin.append(list1)
        all_pinyin.append(list2)
        all_pinyin.append(list3)
        all_pinyin.append(list4)
        return all_pinyin

In [43]:
#将一句话分割成拼音段
def all_pinyin_list(string):
    len_string = len(cut_pinyin(string))
    if len(string[len_string:]) > 0:
        return cut_pinyin(string) + ' ' +  all_pinyin_list(string[len_string:])
    else:
        return cut_pinyin(string)

In [44]:
#找到一句话所有拼音的可能性
def find_pinyin_pro(string):
    pinyin_list = all_pinyin_list(string).split(' ')
    all_pro_list = []
    for i in pinyin_list:
        a = cut_one_pinyin(i)
        all_pro_list.append(a)
    return all_pro_list

In [45]:
import copy

In [46]:
# 对一个拼音的所有可能性纠错
def jiucuo_list(pinyin_list):
    for i in range(len(pinyin_list)):
        for j in range(len(pinyin_list[i])):
            pinyin_list[i][j] = correct(pinyin_list[i][j])
    return pinyin_list

In [68]:
#找到每句话的可能性并进行纠错
def find_sen_pro(string):
    all_pro_list = find_pinyin_pro(string)
    for k in range(len(all_pro_list)-1):
        #放所有的可能性
        pro_list = all_pro_list[0]
        for i in range(len(pro_list[0])):
            pro_list[0][i] = correct(pro_list[0][i])
       #取出下一个拼音的所有可能性
        next_list = all_pro_list[k+1]
        next_list = jiucuo_list(next_list)
        if len(next_list) == 1:
            for i in range(len(pro_list)):
                pro_list[i] = pro_list[i] + next_list[0]
        elif len(next_list) == 2:
            pro_list_1 = copy.deepcopy(pro_list)
            pro_list_2 = copy.deepcopy(pro_list)
            for i in range(len(pro_list_1)):
                pro_list_1[i] = pro_list_1[i] + next_list[0]
            for i in range(len(pro_list_2)):
                pro_list_2[i] = pro_list_2[i] + next_list[1]
            pro_list = pro_list_1 + pro_list_2
        elif len(next_list) == 4:
            pro_list_1 = copy.deepcopy(pro_list)
            pro_list_2 = copy.deepcopy(pro_list)
            pro_list_3 = copy.deepcopy(pro_list)
            pro_list_4 = copy.deepcopy(pro_list)
            for i in range(len(pro_list_1)):
                pro_list_1[i] = pro_list_1[i] + next_list[0]
            for i in range(len(pro_list_2)):
                pro_list_2[i] = pro_list_2[i] + next_list[1]
            for i in range(len(pro_list_3)):
                pro_list_3[i] = pro_list_3[i] + next_list[2]
            for i in range(len(pro_list_4)):
                pro_list_4[i] = pro_list_4[i] + next_list[3]
            pro_list = pro_list_1 + pro_list_2 + pro_list_3 + pro_list_4
        all_pro_list[0] = pro_list
    return all_pro_list[0]

In [70]:
def max_pro_sen(string):
    all_pro_sen_list = find_sen_pro(string)
#     print(all_pro_sen_list)
    #放每个句子概率的列表
    all_pro_num_list = []
    min_len = 1000
    for sen in all_pro_sen_list:
        if len(sen) < min_len:
            min_len = len(sen)
    all_sen_pro = []
    for sen in all_pro_sen_list:
        sen_pro = get_probablity_3_gram(sen[:min_len])
        all_sen_pro.append(sen_pro)
#     print(all_sen_pro)
#     print(all_sen_pro.index(max(all_sen_pro)))
    return all_pro_sen_list[all_sen_pro.index(max(all_sen_pro))]

In [71]:
string = 'woxiangshangqinnghuadaxue'#qinng -> qing

In [72]:
max_pro_sen(string)

['wo', 'xiang', 'shang', 'qing', 'hua', 'da', 'xue']

In [97]:
string = 'xianweiyuzhongguoxinan'#西安位于中国西南 xian -> xi an

In [98]:
max_pro_sen(string)

['xi', 'an', 'wei', 'yu', 'zhong', 'guo', 'xi', 'nan']

In [81]:
string = 'xiannvhaopiaoliang'#仙女好漂亮 xian -> xian

In [82]:
max_pro_sen(string)

['xian', 'nv', 'hao', 'piao', 'liang']

In [107]:
string = 'woxiangshagnqinnghua'# shagn -> shang  qinng ->qing

In [108]:
max_pro_sen(string)

['wo', 'xiang', 'shang', 'qing', 'hua']